<a href="https://colab.research.google.com/github/seyf1elislam/UMS-Linear/blob/main/UMS-Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Univariate Multiscale Decomposition: Improving Linear Models for Univariate Long-term Time Series Forecasting (Pais'24 2024)
## Running UMS-Linear Experiments

In this notebook, we'll go through the steps required to run experiments with the UMS-Linear model, a state-of-the-art approach for univariate long-term time series forecasting. We'll cover the following:

1. Downloading the official UMS-Linear repository from GitHub
2. Obtaining benchmark datasets for time series forecasting tasks
3. Defining the necessary arguments and configurations
4. Importing the training functions from the downloaded code
5. Running example experiments to train the UMS-Linear model

The UMS-Linear model, introduced in the paper "[Univariate Multiscale Decomposition: Improving Linear Models for Univariate Long-term Time Series Forecasting" (Pais'24 2024)](https://github.com/seyf1elislam/UMS-Linear), combines multiscale decomposition and timestamp embedding with linear models to achieve impressive performance on various univariate time series forecasting benchmarks.

By following the steps in this notebook, you'll gain practical experience in setting up and executing experiments with the UMS-Linear model using its official PyTorch implementation. This will provide you with a solid foundation for further exploration and application of the model to your own time series forecasting tasks.


# **Download data & define required functions**

In [ ]:
#@title # **1) download repository and data**
import os

!git clone "https://{github_token}@github.com/seyf1elislam/Forcasting_Memoire_M2.git"
!git clone "https://github.com/seyf1elislam/UMS-Linear"
base_dir = 'UMS-Linear/'
%cd /content/$base_dir
!mkdir dataset
#download data
# from drive======================================
# !pip install -U gdown==4.6.3
# !gdown --folder 1KljxNkMfo5spW05K1QlIyk5ZLQlKfOxm  --output /content/$base_dir/dataset
# !gdown --folder 1e4qvmCW_FJ0XSKCSyn0LA0fHzRpyG8iF  --output /content/$base_dir/dataset
#from github==================================
!wget "https://github.com/hunter123520/Forecasting-Datasets/raw/main/datasets.rar"
!unrar x /content/$base_dir/datasets.rar /content/$base_dir/dataset/

In [ ]:
#@title # **2) Define Arguments**
import os
import torch
import random
import numpy as np
%matplotlib inline

from models import Informer, Autoformer, Transformer, Linear, NLinear,DLinear,UMSLinear
global_model_dict = {
            'Autoformer': Autoformer,
            'Transformer': Transformer,
            'Informer': Informer,
            'DLinear': DLinear,
            'NLinear': NLinear,
            'Linear': Linear,
            "UMSmodel":UMSLinear,
        }


class AttributeDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

def getModelDict(model_class):
  return AttributeDict({"Model":model_class})


args = {
    # basic config
    "is_training":1, #'status'
    "train_only":False, #'perform training on full input dataset without validation and testing'
    "model_id":"test", #'model id'
    "model":"Autoformer", #'model name, options: [Autoformer, Informer, Transformer]'

    # data loader
    "data":"ETTh1", #'dataset type'
    "root_path":"./data/ETT/", #'root path of the data file'
    "data_path":"ETTh1.csv", #'data file'
    "features":"M", #'forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate'
    "target":"OT", #'target feature in S or MS task'
    "freq":"h", #'freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h'
    "checkpoints":'./checkpoints/', #'location of model checkpoints'

    # forecasting task
    "seq_len":96, #'input sequence length'
    "label_len":48, #'start token length'
    "pred_len":96, #'prediction sequence length'

    # Linear / DLinear / Nlinear /UMS linear .
    "individual":False, #treat each variate(channel) individually'

    # Formers
    "embed_type":0, #'0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding'
    "enc_in":7, # 'encoder input size', DLinear with --individual, use this hyperparameter as the number of channels
    "dec_in":7, #'decoder input size'
    "c_out":7, #'output size'
    "d_model":512, #'dimension of model'
    "n_heads":8, #'num of heads'
    "e_layers":2, #'num of encoder layers'
    "d_layers":1, #'num of decoder layers'
    "d_ff":2048, #'dimension of fcn'
    "moving_avg":25, #'window size of moving average'
    "factor":1, #'attn factor'
    "distil":True, #'whether to use distilling in encoder, using this argument means not using distilling'
    "dropout":0.05, #'dropout'
    "embed":"timeF", #'time features encoding, options:[timeF, fixed, learned]'
    "activation":"gelu", #'activation'
    "output_attention":False, #'whether to output attention in ecoder'
    "do_predict":False, #'whether to predict unseen future data'

    # optimization
    "num_workers":10, #'data loader num workers'
    "itr":1, #'experiments times'
    "train_epochs":10, #'train epochs'
    "batch_size":32, #'batch size of train input data'
    "patience":3, #'early stopping patience'
    "learning_rate":0.0001,#'optimizer learning rate'
    "des":"test", #'exp description'
    "loss":"mse", #'loss function'
    "lradj":"type1", #'adjust learning rate'
    "use_amp":False, #'use automatic mixed precision training'

    # GPU
    "use_gpu":True, #'use gpu'
    "gpu":0, #'gpu'
    "use_multi_gpu":False, #'use multiple gpus'
    "devices":"0,1,2,3", #'device ids of multile gpus'
    "test_flop":False #'See utils/tools for usage'
}

#===================================
args = AttributeDict(args)

args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

args.is_training=1
args.root_path="dataset/"
args.data_path="ETTh1.csv"
args.model_id="ETTh1_336_24"
args.data="ETTh1"
args.seq_len=336
args.enc_in=1
args.des='Exp'
args.itr = 1
args.batch_size = 32
args.features = "S"

#=========================
args.learning_rate = 0.005
args.model="DLinear"
args.pred_len=96
#=========================

if args.use_gpu and args.use_multi_gpu:
    args.dvices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

print('Args in experiment:')
print(args)

# ================


In [ ]:
#@title # **3) define loss functions**
from torch import nn
def default_loss(batch_y,outputs,criterion):
  loss = criterion(outputs, batch_y)
  return loss

def diff_loss(batch_y,outputs,criterion):
  diff_data = outputs[:,1:] - outputs[:,:-1]
  diff_data1 = batch_y[:,1:] - batch_y[:,:-1]
  cr = nn.L1Loss()
  loss = criterion(outputs,batch_y) *(1/3) + cr(outputs,batch_y)*(1/3) + cr(diff_data,diff_data1)*(1/3)
  return loss

def halfmae_halfmse_loss(batch_y,outputs,criterion):
    mae = nn.L1Loss()
    mse = nn.MSELoss()
    loss = mae(outputs,batch_y) *(1/2) + mse(outputs,batch_y)*(1/2)
    return loss

# **Training Section**


In [ ]:
#@title **Train Section**
#@markdown   `Note : the parameters are overriding the args variabls` <br/> `the rest parameters can be edited from the args dict`

#@markdown    for multivar data u need to change the enc_in
#@markdown  >`"ETTh2",enc_in =  7` <br/>
#@markdown  >`"exchange_rate", enc_in = 8`  <br/>
#@markdown  >`"weather", enc_in = 21 `<br/>
#@markdown  >`"electricity"=> enc_in = 321`<br/>
#@markdown  >`"traffic"=> enc_in = 862`<br/>
#@markdown  >`"national_illness"=> enc_in = 7 `<br/>
%matplotlib inline
from Training_new.full_train import full_train
# from IPython.utils import io
# with io.capture_output() as output: #this just in case u cant to hide the printing and take only the result
result  =  full_train(
    args,global_model_dict,
    # use_print=False,
    # use_edited_exp=False,#set this false to switch between the original exp_main and true for exp_main1,
    #ModelName=====================================
    # model="NLinear",#this uses the name from the global_model_dict defined above
    model="UMSmodel",
    # model="DLinear",
    # model="Autoformer",
    # model="DLinear", custom_model=DLinear_Core, #this adds custom model to the globalc model_dict
    # model="UMSmodel", custom_model=UMSLinear.Model,
    #Conf=============================================================
    pred_lens=[96],#default [96,192,336,720]
    #Here u setup ur Hyperparameters==================================================
    learning_rate = 0.0005  ,batch_size = 128,seq_len = 512,
    # learning_rate = 0.001  ,batch_size = 128,seq_len = 512,
    #===============
    data="ETTh1",# ETTh2 / ETTm1 / ETTm2
    features = "S",#default "S" for univariate "M" for multi variate
    # enc_in =  1,#in channels default=1
    #=====================================================
    # train_epochs=20,#default is 20 epoch
    patience = 3,#early stoping patience
    #this can define ur custom lose or use on of the func defined above (default_loss,diff_loss,halfmae_halfmse_loss)
    custom_loss = diff_loss
    # custom_loss = default_loss
    # custom_loss = halfmae_halfmse_loss
    # fixed_seed=2024,#default None means random
    )
result